In [28]:
import os
import getpass

os.environ["LLAMA_CLOUD_API_KEY"] = getpass.getpass("LLamaParse API Key:")

In [29]:
os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ_API_KEY:")

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

c:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [5]:
import nest_asyncio

nest_asyncio.apply()

In [26]:
from llama_parse import LlamaParse

In [30]:

parser = LlamaParse(
    result_type="markdown",
    verbose=True,
    language="en",
    num_workers=1,
    premium_mode = True
)

In [31]:
documents = parser.load_data([r"C:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\iesc111.pdf"])

Parsing files: 100%|██████████| 1/1 [02:22<00:00, 142.54s/it]


In [52]:
len(documents)

14

In [ ]:
print(documents[2].text[::])

In [ ]:
print(documents[3].text[::])

In [32]:
from llama_index.llms.groq import Groq

In [33]:
llm = Groq(model="llama3-70b-8192", api_key="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")
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)

In [34]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")


In [9]:
from llama_index.core import Settings


In [35]:
Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
print(resp)

In [10]:
from llama_index.core.node_parser import MarkdownElementNodeParser


In [ ]:
"""
splitter = SentenceSplitter(
    chunk_size=1024,
    chunk_overlap=20,
)
nodes = splitter.get_nodes_from_documents(documents)

nodes"""

In [ ]:
"""
splitter1 = MarkdownNodeParser(
    include_metadata= False,
    include_prev_next_rel= True,
)

nodes_markdown = splitter1.get_nodes_from_documents(documents)

nodes_markdown[1]
"""

In [82]:
node_parser = MarkdownElementNodeParser(llm=llm, num_workers=8)

In [83]:
nodes_markdown_element = node_parser.get_nodes_from_documents(documents=documents)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [86]:
nodes_markdown_element[7]

IndexNode(id_='db060258-4878-4705-be5c-386b4978e71f', embedding=None, metadata={'col_schema': 'Column: State\nType: string\nSummary: None\n\nColumn: Substance\nType: string\nSummary: None\n\nColumn: Speed in m/s\nType: number\nSummary: None'}, excluded_embed_metadata_keys=['col_schema'], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='7a502ce8-cada-4f01-b621-6b8f4ae55c54', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='9d0d77c9411a751597c9f7f9be255d1ae84b92311c71dcf545bdc170e8689b54'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='56f63ebd-6025-4439-8286-993512753375', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d8f6433e378187647f66887e09562443b2f6efd17af31115e71054d502320695'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8ea8291c-cc0c-4de4-a8a9-be49d3a6b694', node_type=<ObjectType.TEXT: '1'>, metadata={'table_df': "{' State   ': {0: ' Solids  ', 1: '         ', 2: '         ', 3: '       

In [87]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes_markdown_element)

In [88]:
index_markdown_element = VectorStoreIndex(nodes=base_nodes+objects)

In [ ]:
%pip install llama-index-postprocessor-flag-embedding-reranker git+https://github.com/FlagOpen/FlagEmbedding.git

In [61]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker


In [89]:
reranker = FlagEmbeddingReranker(
    top_n=3,
    model="BAAI/bge-reranker-large",
)

In [156]:
markdown_element_query_engine = index_markdown_element.as_query_engine(
    similarity_top_k=5,
    verbose=True
)

In [37]:
def setup_parser():
    return LlamaParse(
        result_type="markdown",
        verbose=True,
        language="en",
        num_workers=1,
        premium_mode=True
    )

In [38]:
def create_and_save_index(embed_model, db_path="./chroma_db_new"):
    parser = setup_parser()
    documents = parser.load_data([r"C:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\iesc111.pdf"])
    
    node_parser = MarkdownElementNodeParser(llm=Settings.llm, num_workers=8)
    nodes_markdown_element = node_parser.get_nodes_from_documents(documents=documents)
    base_nodes, objects = node_parser.get_nodes_and_objects(nodes_markdown_element)
    
    # Create or connect to the persistent ChromaDB
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")

    # Create ChromaVectorStore
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create storage context
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    
    # Create index
    index = VectorStoreIndex(nodes=base_nodes+objects, storage_context=storage_context)
    
    return index

In [39]:
create_and_save_index(embed_model)

Parsing files: 100%|██████████| 1/1 [00:17<00:00, 17.39s/it]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
2it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [284]:
eq = """When  a  sound  is  reflected  from  a  distant  object,  an  echo  is
produced.  Let  the  distance  between  the  reflecting  surface
and  the  source  of  sound  production  remains  the  same.  Do
you hear echo sound on a hotter day?"""

response_element1 = markdown_element_query_engine.query(eq)
print(response_element1)

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.59it/s]
2024-09-30 11:40:10,526 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


The distance of the reflecting surface from the source of sound production remains the same. However, the speed of sound increases with an increase in temperature. Therefore, the time taken for the sound to travel to the reflecting surface and back will decrease on a hotter day. Since the time interval between the original sound and the reflected one must be at least 0.1 s to hear a distinct echo, the echo sound may not be heard on a hotter day if the time interval becomes less than 0.1 s.


In [ ]:
eq2 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
response_element2 = markdown_element_query_engine.query(eq2)
print(response_element2)

In [11]:
from llama_index.core.query_engine import RetrieverQueryEngine


In [12]:
from typing import List, Dict, Any
from llama_index.core import VectorStoreIndex
from llama_index.core.base.response.schema import Response
from llama_index.core.schema import NodeWithScore
from llama_index.llms.groq import Groq
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import FunctionTool

In [13]:
import logging 

In [176]:
class WorkflowState:
    def __init__(self, query: str, initial_response: str):
        self.query = query
        self.initial_response = initial_response
        self.additional_info = ""
        self.final_answer = ""
        self.needs_info = False


def check_completeness(state: WorkflowState, llm: Groq) -> WorkflowState:
    prompt = f"""
    Given the following query and initial response, determine if the response contains 
    all necessary information to fully answer the query. If not, specify what additional 
    information is needed.

    Query: {state.query}
    Initial Response: {state.initial_response}

    Output your response in the following format:
    Complete: [Yes/No]
    Additional Info Needed: [If No, specify what additional information is needed]
    """

    response = llm.complete(prompt)

    # Extract 'Complete' and 'Additional Info Needed' fields from response
    lines = response.split('\n')
    for line in lines:
        if 'Complete' in line:
            state.needs_info = 'No' in line
        elif 'Additional Info Needed' in line and state.needs_info:
            state.additional_info = line.split(': ')[1].strip()

    return state


def retrieve_additional_info(state: WorkflowState, query_engine: RetrieverQueryEngine) -> WorkflowState:
    if state.needs_info:
        additional_info_query = state.additional_info
        response = query_engine.query(additional_info_query)

        if isinstance(response, str):
            state.additional_info = response.strip()
        else:
            raise TypeError("Expected string response from query engine.")
    
    return state


def generate_final_answer(state: WorkflowState, llm: Groq) -> WorkflowState:
    prompt = f"""
    Given the following query, initial response, and additional information (if any),
    generate a complete and accurate answer.

    Query: {state.query}
    Initial Response: {state.initial_response}
    Additional Information: {state.additional_info}

    Complete Answer:
    """

    response = llm.complete(prompt)

    if isinstance(response, str):
        state.final_answer = response.strip()
    else:
        raise TypeError("Expected string response from generate_final_answer.")

    return state


def create_workflow(query_engine: RetrieverQueryEngine, llm: Groq):
    tools = [
        FunctionTool.from_defaults(
            fn=lambda x: check_completeness(x, llm),
            name="check_completeness",
            description="Check if the initial response is complete"
        ),
        FunctionTool.from_defaults(
            fn=lambda x: retrieve_additional_info(x, query_engine),
            name="retrieve_additional_info",
            description="Retrieve additional information if needed"
        ),
        FunctionTool.from_defaults(
            fn=lambda x: generate_final_answer(x, llm),
            name="generate_final_answer",
            description="Generate the final answer"
        )
    ]

    agent = OpenAIAgent.from_tools(
        tools,
        llm=llm,
        verbose=True
    )

    return agent


def solve_query_with_workflow(query: str, query_engine: RetrieverQueryEngine, llm: Groq) -> str:
    # Get initial response
    initial_response = query_engine.query(query)

    # Create initial state
    state = WorkflowState(query=query, initial_response=str(initial_response))

    # Create and run the workflow
    agent = create_workflow(query_engine, llm)

    # Define the task for the agent
    task = f"""
    Given the following query and initial response, use the available tools to ensure a complete and accurate answer is provided.
    If the initial response is incomplete, retrieve additional information and generate a final answer.

    Query: {state.query}
    Initial Response: {state.initial_response}

    Use the tools in this order:
    1. check_completeness
    2. retrieve_additional_info (if needed)
    3. generate_final_answer

    Return the final answer.
    """

    # Run the agent
    response = agent.chat(task)

    # Process the response to extract the final answer
    if hasattr(response, 'response'):
        final_answer = response.response.strip()
    else:
        raise ValueError("Unexpected response format from agent.")

    return final_answer

In [ ]:
query = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
#logger.info(f"Processing query: {query}")
final_answer = solve_query_with_workflow(query, markdown_element_query_engine, llm)
#logger.info(f"Final answer: {final_answer}")
print(final_answer)

In [ ]:
eq2 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."
final_answer = solve_query_with_workflow(eq2, markdown_element_query_engine, llm)

print(final_answer)

In [188]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [14]:
from typing import List, Dict
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.query_engine import BaseQueryEngine
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core.node_parser import MarkdownElementNodeParser
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_parse import LlamaParse

In [24]:
from llama_parse import LlamaParse

In [15]:
import re
from typing import Dict, Tuple


In [228]:
class QueryProcessor:
    def __init__(self, query_engine: BaseQueryEngine):
        self.query_engine = query_engine

    def process_query(self, query: str) -> Dict[str, str]:
        response = self.query_engine.query(query)
        return {
            "response": response.response,
            "source_nodes": [node.node.get_content() for node in response.source_nodes],
        }

class InformationAnalyzer:
    def __init__(self, groq_llm):
        self.llm = groq_llm

    def analyze_response(self, initial_response: str, query: str) -> str:
        messages = [
            ChatMessage(role="system", content="""You are an AI assistant tasked with analyzing responses and determining if additional information is needed. 
            Request additional information if:
            1. The query cannot be fully answered without additional specific data.
            2. The query asks for a specific numerical value, and the initial response doesn't provide it.
            3. The initial response uses variables or placeholders without providing their actual values.
            If additional information is needed, specify it as a simple, direct question."""),
            ChatMessage(role="user", content=f"""
                Given the following query and initial response, determine if any additional information is required to fully answer the query. 
                If the initial response fully answers the query with all necessary specific values, state that no additional information is required.
                If additional information is necessary, specify what information is needed as a simple, direct question.

                Query: {query}
                Initial Response: {initial_response}

                Is additional information required? If no, state "No additional information is required." If yes, what specific information is needed? Phrase it as a simple question.
            """)
        ]
        response = self.llm.chat(messages)
        return response.message.content

    def formulate_final_query(self, initial_response: str, additional_info: str, query: str) -> str:
        messages = [
            ChatMessage(role="system", content="You are an AI assistant tasked with formulating queries based on given information. Do not use approximations or assumptions."),
            ChatMessage(role="user", content=f"""
                Given the following:
                1. Original query: {query}
                2. Initial response: {initial_response}
                3. Additional information: {additional_info}

                Formulate a new query that will help generate a complete and accurate final answer. Do not use approximations or assumptions.
            """)
        ]
        response = self.llm.chat(messages)
        return response.message.content
    

    def generate_final_answer(self, query: str, initial_response: str, additional_info: str, final_query_response: str) -> str:
        messages = [
            ChatMessage(role="system", content="You are an AI assistant tasked with generating comprehensive answers based on given information. Stick to the information provided and do not introduce unrelated concepts or data."),
            ChatMessage(role="user", content=f"""
                Given the following information:
                1. Original query: {query}
                2. Initial response: {initial_response}
                3. Additional information gathered: {additional_info}
                4. Response to final query: {final_query_response}

                Please generate a complete and accurate final answer to the original query, incorporating all relevant information. Do not introduce concepts or data not mentioned in the provided information.
            """)
        ]
        response = self.llm.chat(messages)
        return response.message.content

def create_rag_agent(index: VectorStoreIndex, llm: Groq) -> Tuple[BaseQueryEngine, Groq]:
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        node_postprocessors=[
            FlagEmbeddingReranker(
                top_n=3,
                model="BAAI/bge-reranker-large",
            )
        ],
        verbose=True
    )
    return query_engine, llm

def process_advanced_query(query_engine: BaseQueryEngine, llm: Groq, query: str) -> str:
    processor = QueryProcessor(query_engine)
    analyzer = InformationAnalyzer(llm)

    # Step 1: Generate initial response
    initial_result = processor.process_query(query)
    initial_response = initial_result['response']
    print(f"Initial Response: {initial_response}")

    # Step 2: Check if additional information is required
    analysis_result = analyzer.analyze_response(initial_response, query)
    print(f"Analysis Result: {analysis_result}")

    if "no additional information is required" in analysis_result.lower():
        return initial_response

    # Step 3: Fetch additional required information
    additional_info_result = processor.process_query(analysis_result)
    additional_info = additional_info_result['response']
    print(f"Additional Information: {additional_info}")

    # Step 4: Formulate new query and generate final answer
    final_query = analyzer.formulate_final_query(initial_response, additional_info, query)
    print(f"Final Query: {final_query}")
    
    final_query_result = processor.process_query(final_query)
    final_query_response = final_query_result['response']
    print(f"Final Query Response: {final_query_response}")

    # Step 5: Generate comprehensive final answer
    final_answer = analyzer.generate_final_answer(query, initial_response, additional_info, final_query_response)
    return final_answer

In [ ]:
try:
    agent = create_rag_agent(index_markdown_element, llm)
    
    ques = "Two children are at opposite ends of an steel rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in steel to reach the second child."
    result = process_advanced_query(agent, ques)
    print(f"Final Query Result: {result}")
except Exception as e:
    logger.error(f"An error occurred in the main execution: {str(e)}")
    print(f"An error occurred: {str(e)}")

In [229]:
query_engine, llm = create_rag_agent(index_markdown_element, llm)

my_question = "Two children are at opposite ends of a steel rod and an aluminium rod. One strikes the end of each rod with a stone. Find the ratio of times taken by the sound wave in aluminium and in steel to reach the second child."

result = process_advanced_query(query_engine, llm, my_question)
print(f"\nFinal Answer: {result}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]
2024-09-29 10:54:29,375 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Initial Response: The speed of sound in a medium is a characteristic of that medium. Since the speed of sound in aluminium and steel are different, the time taken by the sound wave to reach the second child will also be different.

Let's assume the length of each rod is L. Let the speed of sound in aluminium be v_al and in steel be v_st. 

The time taken by the sound wave in aluminium to reach the second child is t_al = L / v_al. 

The time taken by the sound wave in steel to reach the second child is t_st = L / v_st.

The ratio of times taken by the sound wave in aluminium and in steel to reach the second child is t_al / t_st = (L / v_al) / (L / v_st) = v_st / v_al.

This ratio depends on the speeds of sound in aluminium and steel, which are characteristic properties of these materials.


2024-09-29 10:54:30,018 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Analysis Result: No additional information is required.

Final Answer: The speed of sound in a medium is a characteristic of that medium. Since the speed of sound in aluminium and steel are different, the time taken by the sound wave to reach the second child will also be different.

Let's assume the length of each rod is L. Let the speed of sound in aluminium be v_al and in steel be v_st. 

The time taken by the sound wave in aluminium to reach the second child is t_al = L / v_al. 

The time taken by the sound wave in steel to reach the second child is t_st = L / v_st.

The ratio of times taken by the sound wave in aluminium and in steel to reach the second child is t_al / t_st = (L / v_al) / (L / v_st) = v_st / v_al.

This ratio depends on the speeds of sound in aluminium and steel, which are characteristic properties of these materials.


In [225]:
another_question = """What is the range of frequencies associated with:
(a) Infrasound?
(b) Ultrasound?"""
result = process_advanced_query(query_engine, llm, another_question)
print(f"\nFinal Answer: {result}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 13.96it/s]
2024-09-29 09:47:10,528 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Initial Response: (a) Infrasound: Frequencies below 20 Hz
(b) Ultrasound: Frequencies above 20 kHz


2024-09-29 09:47:10,885 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Analysis Result: No additional information is required.

Final Answer: (a) Infrasound: Frequencies below 20 Hz
(b) Ultrasound: Frequencies above 20 kHz


In [226]:
new_ques = "Give two practical applications of reflection of sound waves."
new_result = process_advanced_query(query_engine, llm, new_ques)
print(f"\nFinal Answer: {new_result}")

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]
2024-09-29 09:51:33,399 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Initial Response: Two practical applications of reflection of sound waves are ultrasonography, which is used for medical imaging and examination of the foetus during pregnancy, and breaking small 'stones' formed in the kidneys into fine grains that can be flushed out with urine.


2024-09-29 09:51:33,794 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Analysis Result: No additional information is required.

Final Answer: Two practical applications of reflection of sound waves are ultrasonography, which is used for medical imaging and examination of the foetus during pregnancy, and breaking small 'stones' formed in the kidneys into fine grains that can be flushed out with urine.


In [16]:
from typing import Dict, List, Tuple
from llama_index.core import VectorStoreIndex
from llama_index.core.query_engine import BaseQueryEngine
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker
from llama_index.core.schema import NodeWithScore
from llama_index.llms.groq import Groq
from llama_index.core.llms import ChatMessage

In [ ]:
class QueryEngine:
    def __init__(self, index: VectorStoreIndex):
        self.query_engine = index.as_query_engine(
            similarity_top_k=5,
            node_postprocessors=[
                FlagEmbeddingReranker(
                    top_n=3,
                    model="BAAI/bge-reranker-large",
                )
            ],
            verbose=True
        )

    def query(self, query: str) -> Dict[str, str]:
        print(f"QueryEngine: Executing query: {query}")
        response = self.query_engine.query(query)
        print(f"QueryEngine: Response received")
        return {
            "response": response.response,
            "source_nodes": [node.node.get_content() for node in response.source_nodes],
        }

In [ ]:
class AdvancedQueryAgent:
    def __init__(self, query_engine: QueryEngine, llm: Groq):
        self.query_engine = query_engine
        self.llm = llm

    def analyze_response(self, initial_response: str, query: str) -> str:
        print(f"AdvancedQueryAgent: Analyzing response for query: {query}")
        messages = [
            ChatMessage(role="system", content="Analyze the response and determine if additional information is needed. If so, specify it as a simple, direct question without using any variables or abbreviations. Use natural language to ask for the required information."),
            ChatMessage(role="user", content=f"Query: {query}\nInitial Response: {initial_response}\n\nIs additional information required? If yes, what specific information is needed? Please provide the follow-up question in simple, clear language.")
        ]
        response = self.llm.chat(messages)
        print(f"AdvancedQueryAgent: Analysis complete")
        return response.message.content

    def get_additional_info(self, analysis: str) -> Dict[str, str]:
        print(f"AdvancedQueryAgent: Getting additional info based on analysis: {analysis}")
        return self.query_engine.query(analysis)

    def generate_final_answer(self, query: str, initial_response: str, additional_info: str) -> str:
        print(f"AdvancedQueryAgent: Generating final answer for query: {query}")
        messages = [
            ChatMessage(role="system", content="Generate a comprehensive final answer based on the given information."),
            ChatMessage(role="user", content=f"Original query: {query}\nInitial response: {initial_response}\nAdditional information: {additional_info}\n\nPlease provide a complete and accurate final answer.")
        ]
        response = self.llm.chat(messages)
        print(f"AdvancedQueryAgent: Final answer generated")
        return response.message.content

In [236]:

class RAGSystem:
    def __init__(self, index: VectorStoreIndex, llm: Groq):
        self.query_engine = QueryEngine(index)
        self.advanced_agent = AdvancedQueryAgent(self.query_engine, llm)

    def basic_query(self, query: str) -> str:
        print(f"RAGSystem: Executing basic query: {query}")
        result = self.query_engine.query(query)
        print(f"RAGSystem: Basic query complete")
        return result["response"]

    def advanced_query(self, query: str, initial_response: str) -> str:
        print(f"RAGSystem: Starting advanced query process for: {query}")
        analysis = self.advanced_agent.analyze_response(initial_response, query)
        print(f"RAGSystem: Analysis result: {analysis}")
        
        if "no additional information is required" in analysis.lower():
            print("RAGSystem: No additional information required. Returning initial response.")
            return initial_response

        additional_info = self.advanced_agent.get_additional_info(analysis)
        print(f"RAGSystem: Additional info retrieved: {additional_info['response']}")
        
        final_answer = self.advanced_agent.generate_final_answer(query, initial_response, additional_info["response"])
        print("RAGSystem: Advanced query process complete")
        return final_answer

def main(index: VectorStoreIndex, llm: Groq):
    rag_system = RAGSystem(index, llm)

    while True:
        query = input("Enter your query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        print("\n--- Starting Basic Query ---")
        initial_response = rag_system.basic_query(query)
        print(f"\nInitial response: {initial_response}\n")

        user_choice = input("Is this answer sufficient? (yes/no): ")
        if user_choice.lower() == 'no':
            print("\n--- Starting Advanced Query ---")
            final_answer = rag_system.advanced_query(query, initial_response)
            print(f"\nFinal answer: {final_answer}\n")
        else:
            print("Great! Let me know if you have any more questions.")


In [241]:
if __name__ == "__main__":
    # Assume index and llm are already initialized outside this script
    # You would typically import them or pass them as arguments
    main(index_markdown_element, llm)


--- Starting Basic Query ---
RAGSystem: Executing basic query: what sound does the cow makes ? 
QueryEngine: Executing query: what sound does the cow makes ? 


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.94it/s]
2024-09-30 02:53:44,158 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


QueryEngine: Response received
RAGSystem: Basic query complete

Initial response: There is no information about the sound a cow makes in the provided context. The context discusses sound waves, their properties, and how they are produced and propagated, but it does not mention the specific sound made by a cow.

Great! Let me know if you have any more questions.


In [243]:
from typing import Dict, List, Any
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step, Event
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.llms import MessageRole
from llama_index.core.tools import FunctionTool
from groq import Groq

In [41]:
from typing import Dict, Any, Optional
import serpapi
from getpass import getpass
import os

In [42]:
from serpapi.google_search import GoogleSearch

In [268]:
SUMMARY_TEXT = """Concise summary:
This document covers the fundamentals of sound, including:
Production and propagation of sound waves
Wave characteristics: wavelength, frequency, amplitude, speed
Sound reflection, echo, and reverberation
Speed of sound in various media
Human hearing range (20 Hz - 20 kHz)
Infrasound and ultrasound
Applications of ultrasound in medicine and industry
Key concepts include vibrating objects as sound sources, longitudinal waves, compressions and rarefactions, reflection laws, and factors affecting sound speed and perception. The text provides a comprehensive introduction to acoustics suitable for high school students."""

In [290]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Load a model for similarity calculation

summary_text = """
The document covers sound wave production, propagation, wavelength, frequency, amplitude, 
sound speed, reflection, echo, reverberation, speed of sound in various media, human hearing 
range (20 Hz - 20 kHz), infrasound, ultrasound, and applications in medicine and industry. 
It also discusses vibrating objects, longitudinal waves, compressions, rarefactions, reflection laws, 
and factors affecting sound speed and perception. The text introduces acoustics for high school students.
"""

summary_embedding = model.encode(summary_text, convert_to_tensor=True)

2024-10-01 03:12:48,484 - INFO - Load pretrained SentenceTransformer: paraphrase-MiniLM-L6-v2
c:\Users\nisha\Desktop\RAG_Assign\RAG_Assign\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-10-01 03:13:18,466 - INFO - Use pytorch device_name: cpu
Batches: 100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


In [291]:
"""def is_query_related_to_context(query: str) -> bool:
    """#Determines if the query relates to the acoustic context using semantic similarity."""
    #query_embedding = model.encode(query, convert_to_tensor=True)
    
    #similarity_score = util.pytorch_cos_sim(query_embedding, summary_embedding).item()
    #print(f"Similarity score for query '{query}': {similarity_score}")

    # Set a similarity threshold, e.g., 0.5 to determine relevance
    #return similarity_score > 0.5"""

In [304]:
def is_query_related_to_context(query: str) -> bool:
    return any(keyword.lower() in query.lower() for keyword in [
        "sound", "wave", "frequency", "amplitude", "reflection", 
        "ultrasound", "medicine", "industry", "echo", "reverberation", 
        "human hearing", "infrasound"
    ])

In [49]:
"""def is_query_related_to_context(query: str) -> bool:
    return any(keyword.lower() in query.lower() for keyword in [
        "sound", "wave", "frequency", "amplitude", "reflection", 
        "ultrasound", "medicine", "industry", "echo", "reverberation", 
        "human hearing", "infrasound"
    ])"""

serpapi_params = {
    "engine": "google",
    "api_key": getpass("SerpAPI_key: ")
}

In [ ]:
# Event definitions
class InputEvent(Event):
    input: str

class ToolCallEvent(Event):
    id: str
    name: str
    params: Dict[str, Any]

class UserDecisionEvent(Event):
    decision: str

In [ ]:
# Tool functions
async def query_engine(index: VectorStoreIndex, query: str) -> Dict[str, Any]:
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        verbose=True
    )
    print(f"QueryEngine: Executing query: {query}")
    response = query_engine.query(query)
    return {
        "response": response.response,
        "source_nodes": [node.node.get_content() for node in response.source_nodes],
    }

async def analyze_response(llm: Groq, initial_response: str, query: str) -> Dict[str, Any]:
    print(f"AdvancedQueryAgent: Analyzing response for query: {query}")
    messages = [
        ChatMessage(role="system", content="Analyze the initial_response and determine if additional information is needed based in what the query is asking. If so, specify it as a simple, direct question without using any variables or abbreviations. Use natural language to ask for the required information."),
        ChatMessage(role="user", content=f"Query: {query}\nInitial Response: {initial_response}\n\nIs additional information required? If yes, what specific information is needed? Please provide the follow-up question in simple, clear language.")
    ]
    response = llm.chat(messages)
    return {"analysis": response.message.content}

async def generate_final_answer(llm: Groq, query: str, initial_response: str, additional_info: str) -> Dict[str, Any]:
    print(f"AdvancedQueryAgent: Generating final answer for query: {query}")
    messages = [
        ChatMessage(role="system", content="Generate a comprehensive final answer based on the given information."),
        ChatMessage(role="user", content=f"Original query: {query}\nInitial response: {initial_response}\nAdditional information: {additional_info}\n\nPlease provide a complete and accurate final answer.")
    ]
    response = llm.chat(messages)
    return {"final_answer": response.message.content}


async def web_search_tool(query: str) -> Dict[str, Any]:
    print(f"Web Search Tool: Searching for query: {query}")
    
    search = GoogleSearch({
        **serpapi_params,
        "q": query,
        "num": 5
    })
    
    results = search.get_dict().get("organic_results", [])
    
    if results:
        contexts = "\n---\n".join(
            [f"Title: {x['title']}\nSnippet: {x['snippet']}\nLink: {x['link']}" for x in results]
        )
    else:
        contexts = "No results found."
    
    return {
        "response": contexts,
        "source": "Web"
    }

async def handle_human_message(llm: Groq, message: str) -> Dict[str, Any]:
    """Handles generic human-like interactions like greetings."""
    print(f"LLM Agent: Handling human message: {message}")
    
    messages = [
        ChatMessage(role="system", content="Respond as a friendly assistant. If the message seems like a greeting or informal question, reply accordingly."),
        ChatMessage(role="user", content=f"Message: {message}")
    ]
    
    response = llm.chat(messages)
    return {"response": response.message.content}



async def classify_query(llm: Groq, query: str) -> str:
    """Classifies the query into greeting, index_search, or web_search."""
    
    print(f"LLM Router: Classifying query: {query}")
    
    # First, check if the query is a greeting using the LLM
    messages = [
        ChatMessage(role="system", content=""" You are an expert in routing user queries to the appropriate category: 'greeting', 'index_search', or 'web_search'.

        If the user's message is a greeting or a casual interaction, categorize it as 'greeting'.
        Use 'index_search' for queries related to sound and its properties. This includes but is not limited to:
        Production and propagation of sound waves
        Wave characteristics (frequency, amplitude, etc.)
        Sound reflection, echo, and reverberation
        Speed of sound in different media
        Human hearing and perception of sound
        Infrasound, ultrasound, and their applications (e.g., medical, industrial)
        You do not need to rely strictly on keywords but should focus on whether the query is broadly related to sound and its scientific aspects.
        For all other queries that don’t fit these categories, choose 'web_search'.

        Your only response should be one of the following: 'greeting', 'index_search', or 'web_search'.
"""),
        ChatMessage(role="user", content=f"Query: {query}")
    ]
    
    response = llm.chat(messages)
    classification = response.message.content.strip().lower()
    print(classification)
    return classification
    

In [352]:
# Main RAG System as a Workflow
class RAGSystem(Workflow):
    def __init__(
        self,
        *args: Any,
        index: VectorStoreIndex,
        llm: Groq,
        timeout: int = 20,
    ):
        super().__init__(*args)
        self._timeout = timeout
        self.llm = llm
        self.index = index

        # Ensure the tools have the correct functions and metadata
        self.tools = {
            "query_engine": FunctionTool(
                fn=lambda **params: query_engine(self.index, **{k: v for k, v in params.items() if k != "index"}),
                #fn=lambda query: query_engine(self.index, query=query),
                metadata={"name": "query_engine", "description": "Fetches results from the vector index based on the query."}
            ),
            "analyze_response": FunctionTool(
                fn=lambda **params: analyze_response(self.llm, **params),
                metadata={"name": "analyze_response", "description": "Analyzes the response and determines if more information is needed."}
            ),
            "generate_final_answer": FunctionTool(
                fn=lambda **params: generate_final_answer(self.llm, **params),
                metadata={"name": "generate_final_answer", "description": "Generates the final answer based on the analysis."}
            ),
            "web_search_tool": FunctionTool(  # Add this for clarity and ensure it is referenced
                fn=lambda **params: web_search_tool(params["query"]),
                metadata={"name": "web_search_tool", "description": "Searches the web for the query."}
            )
        }

    @step
    async def handle_initial_query(self, ev: InputEvent, context: Dict[str, Any]) -> StopEvent | ToolCallEvent:
        query = ev.input
        context["original_query"] = query  # Store the query in context


        classification = await classify_query(self.llm, query)

        if classification == "greeting":
            print("\n--- Detected a human message. Invoking LLM agent ---")
            response = await handle_human_message(self.llm, query)
            print(f"LLM response: {response['response']}")
            return StopEvent(result={"message": response['response']})

        elif classification == "index_search":
            return ToolCallEvent(
                id="initial_query",
                name="query_engine",
                params={"index": self.index, "query": query},
            )
        else :
            print("\n--- Query is not related to the context. Initiating web search ---")
            return ToolCallEvent(
                id="web_search",
                name="web_search_tool",
                params={"query": query}
            )

    @step
    async def process_initial_response(self, ev: ToolCallEvent, context: Dict[str, Any]) -> StopEvent | UserDecisionEvent:
        
        if isinstance(ev, StopEvent):
            print("Received StopEvent, no further processing needed.")
            return ev
        if ev.id =="web_search":
            tool_output = await self.tools[ev.name].fn(**ev.params)  # Call the appropriate tool function
            initial_response = tool_output["response"]
            context["initial_response"] = initial_response
            print(f"\nInitial Response: {initial_response}")  # Print the initial response
            return StopEvent(result={"message": "Web search results provided."})

        if ev.id == "initial_query":
            tool_output = await self.tools[ev.name].fn(**ev.params)  # Call the appropriate tool function
            initial_response = tool_output["response"]
            context["initial_response"] = initial_response
            print(f"\nInitial Response: {initial_response}")  # Print the initial response

            decision = input("Is this response satisfactory? (yes/no): ").strip().lower()
            return UserDecisionEvent(decision=decision)

    @step
    async def handle_user_decision(self, ev: UserDecisionEvent, context: Dict[str, Any]) -> StopEvent | ToolCallEvent:
        try:
            
            if ev.decision == 'yes':
                print("\nThank you for confirming. Exiting the program.")
                return StopEvent(result={"message": "User confirmed satisfactory response."})  # Exit with message
            
            else:
                print("\n--- Starting Advanced Query ---")
                return ToolCallEvent(
                    id="analyze_response",
                    name="analyze_response",
                    params={
                        "initial_response": context["initial_response"], 
                        "query": context["original_query"]
                    },
                )
        except Exception as e:
            print(f"Error in handling user decision: {e}")
            return StopEvent(result={"message": "An error occurred in user decision handling."})

    @step
    async def process_analysis(self, ev: ToolCallEvent, context: Dict[str, Any]) -> ToolCallEvent:
        analysis_output = await self.tools[ev.name].fn(**ev.params)
        analysis = analysis_output["analysis"]
        print(f"RAGSystem: Analysis result: {analysis}")
        context["follow_up_query"] = analysis  # Store the follow-up query in context
        
        return ToolCallEvent(
            id="additional_query",
            name="query_engine",
            params={"query": analysis},  # Pass follow-up query
        )

    @step
    async def fetch_additional_info(self, ev: ToolCallEvent, context: Dict[str, Any]) -> ToolCallEvent:
        additional_info = await self.tools[ev.name].fn(**ev.params)
        print(f"RAGSystem: Additional info retrieved: {additional_info['response']}")
        context["additional_info"] = additional_info["response"]  # Store the additional info in context
        
        return ToolCallEvent(
            id="generate_final_answer",
            name="generate_final_answer",
            params={
                "query": context["original_query"],  # Fetch original query from context
                "initial_response": context["initial_response"],  # Fetch initial response from context
                "additional_info": additional_info["response"]
            }
        )

    @step
    async def generate_final_answer(self, ev: ToolCallEvent, context: Dict[str, Any]) -> StopEvent:
        final_answer = await self.tools[ev.name].fn(**ev.params)
        return StopEvent(result={"response": final_answer["final_answer"]})


async def main(index: VectorStoreIndex, llm: Groq):
    rag_system = RAGSystem(index=index, llm=llm)

    while True:
        query = input("Enter your query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        # Initialize context
        context = {}

        print("\n--- Starting Query Process ---")
        
        initial_query_event = await rag_system.handle_initial_query(InputEvent(input=query), context)
        if isinstance(initial_query_event,StopEvent):
            print(f"\nFinal message: {initial_query_event.result['message']}\n")
            continue

        process_initial_response_event = await rag_system.process_initial_response(initial_query_event, context)
        if isinstance(process_initial_response_event, StopEvent):
            print(f"\nFinal message: {process_initial_response_event.result['message']}\n")
            continue
        
        # Pass context through the workflow
        user_decision_event = await rag_system.handle_user_decision(process_initial_response_event, context)
        if isinstance(user_decision_event, StopEvent):
            print(f"\nFinal message: {user_decision_event.result['message']}\n")
            continue
            
        else:
            analysis_event = await rag_system.process_analysis(user_decision_event, context)
            additional_info_event = await rag_system.fetch_additional_info(analysis_event, context)
            final_answer_event = await rag_system.generate_final_answer(additional_info_event, context)
            print(f"\nFinal answer: {final_answer_event.result['response']}\n")

if __name__ == "__main__":
    # Assume index and llm are already initialized outside this script
    # You would typically import them or pass them as arguments
    import asyncio
    asyncio.run(main(index_markdown_element, llm))


--- Starting Query Process ---
LLM Router: Classifying query: how does sound propogaes ? 


2024-10-02 11:54:37,277 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


index_search
QueryEngine: Executing query: how does sound propogaes ? 


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.51it/s]
2024-10-02 11:54:38,312 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"



Initial Response: Sound propagates through a medium, such as air, water, or solid objects, in the form of a wave. When an object vibrates, it sets the particles of the medium around it vibrating. These particles then exert a force on adjacent particles, causing them to displace from their equilibrium position. This process continues, allowing the disturbance to travel through the medium. The particles of the medium do not move forward themselves, but the disturbance is carried forward, allowing the sound wave to propagate.


TypeError: object NoneType can't be used in 'await' expression

In [308]:
text = """"You are an expert in routing user queries to the appropriate category: 'greeting', 'index_search', or 'web_search'.

If the user's message is a greeting or a casual interaction, categorize it as 'greeting'.
Use 'index_search' for queries related to sound and its properties. This includes but is not limited to:
Production and propagation of sound waves
Wave characteristics (frequency, amplitude, etc.)
Sound reflection, echo, and reverberation
Speed of sound in different media
Human hearing and perception of sound
Infrasound, ultrasound, and their applications (e.g., medical, industrial)
You do not need to rely strictly on keywords but should focus on whether the query is broadly related to sound and its scientific aspects.
For all other queries that don’t fit these categories, choose 'web_search'.

Your only response should be one of the following: 'greeting', 'index_search', or 'web_search'."
"""

In [125]:
"""from typing import List
from pydantic import Field
from langchain.schema import BaseRetriever, Document
from llama_index.core import VectorStoreIndex
from langgraph.graph import StateGraph, END
from langchain_core.language_models import BaseLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from typing import Dict, Any
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field
from typing import Optional
from langgraph.graph import StateGraph, END
from langchain_core.language_models import BaseLLM
from langchain_core.messages import BaseMessage

from typing import Any, List, Optional
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models import BaseLLM
from langchain_core.outputs import Generation, LLMResult

from pydantic import BaseModel, Field
from typing import Any, List, Optional
from llama_index.llms.groq import Groq
from langchain.schema import HumanMessage

#from llama_index.llm_predictor.base import BaseLLM

#from llama_index.output_parsers.llm import LLMResult, Generation
#from llama_index.callbacks.base import CallbackManagerForLLMRun

# Wrapper class to adapt LlamaIndex VectorStoreIndex to LangChain BaseRetriever interface
class LlamaIndexVectorStoreRetriever(BaseRetriever):
    vector_store_index: VectorStoreIndex = Field(description="The LlamaIndex VectorStoreIndex")
    similarity_top_k: int = Field(default=5, description="Number of top similar documents to retrieve")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        retriever = self.vector_store_index.as_retriever(similarity_top_k=self.similarity_top_k)
        nodes = retriever.retrieve(query)
        return [Document(page_content=node.node.get_content(), metadata=node.node.metadata) for node in nodes]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement async version if needed
        return self._get_relevant_documents(query)"""

"""class State:
    #The state of the workflow.
    def __init__(
        self,
        query: str,
        initial_response: str,
        additional_info: str = "",
        final_answer: str = "",
        needs_info: bool = False
    ):
        self.query = query
        self.initial_response = initial_response
        self.additional_info = additional_info
        self.final_answer = final_answer
        self.needs_info = needs_info

class WorkflowState(BaseModel):
    query: str
    initial_response: str
    additional_info: str = ""
    final_answer: str = ""
    needs_info: bool = False


class GroqLLMWrapper(BaseLLM, BaseModel):
    groq_llm: Groq = Field(..., description="The Groq LLM instance")

    def _generate(
        self,
        prompts: List[str],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> LLMResult:
        generations = []
        for prompt in prompts:
            response = self.groq_llm.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                model="llama3-70b-8192",  # Already passed in Groq instance
                stop=stop,
                **kwargs
            )
            generations.append([Generation(text=response.choices[0].message.content)])
        return LLMResult(generations=generations)

    @property
    def _llm_type(self) -> str:
        return "groq"

    def predict_messages(self, messages: List[str], **kwargs) -> str:
    # Convert HumanMessage objects to their text content
        prompt = " ".join([msg.content if isinstance(msg, HumanMessage) else msg for msg in messages])
        response = self._generate([prompt], **kwargs)
        return response.generations[0][0].text"""


#def check_completeness(state: WorkflowState, llm: GroqLLMWrapper) -> Dict[str, WorkflowState]:
#    prompt = ChatPromptTemplate.from_template(
#        """Given the following query and initial response, determine if the response contains 
"""        all necessary information to fully answer the query. If not, specify what additional 
        information is needed.

        Query: {query}
        Initial Response: {initial_response}

        Output your response in the following format:
        Complete: [Yes/No]
        Additional Info Needed: [If No, specify what additional information is needed]
        """
#    )
    
#    parser = StructuredOutputParser.from_response_schemas([
#        ResponseSchema(name="Complete", description="Whether the response is complete (Yes/No)"),
#        ResponseSchema(name="Additional Info Needed", description="What additional information is needed, if any")
#    ])
    
#    response = llm.predict_messages(prompt.format_messages(
#        query=state.query, 
#        initial_response=state.initial_response
#    ))
    
#    parsed_response = parser.parse(response)
#    state.needs_info = parsed_response["Complete"].lower() == "no"
#    if state.needs_info:
#        state.additional_info = parsed_response["Additional Info Needed"]
    
#    return {"state": state}

"""def retrieve_additional_info(state: WorkflowState, retriever: LlamaIndexVectorStoreRetriever) -> Dict[str, WorkflowState]:
    #Retrieve additional information if needed.
    if state.needs_info:
        additional_info_query = f"Provide information about: {state.additional_info}"
        docs = retriever.get_relevant_documents(additional_info_query)
        state.additional_info = "\n".join([doc.page_content for doc in docs])
    return {"state": state}"""

#def generate_final_answer(state: WorkflowState, llm: GroqLLMWrapper) -> Dict[str, WorkflowState]:
#    prompt = ChatPromptTemplate.from_template(
#        """Given the following query, initial response, and additional information (if any),
#        generate a complete and accurate answer.0

#        Query: {query}
#        Initial Response: {initial_response}
 #       Additional Information: {additional_info}

#        Complete Answer:
#        """
#    )
    
#    state.final_answer = llm.predict_messages(prompt.format_messages(
#        query=state.query,
#        initial_response=state.initial_response,
#        additional_info=state.additional_info
#    ))
    
#    return {"state": state}

"""def create_workflow(llm: BaseLLM, retriever: LlamaIndexVectorStoreRetriever) -> StateGraph:
    workflow = StateGraph(WorkflowState)
    
    # Add nodes
    workflow.add_node("check_completeness", lambda state: check_completeness(state, llm))
    workflow.add_node("retrieve_additional_info", lambda state: retrieve_additional_info(state, retriever))
    workflow.add_node("generate_final_answer", lambda state: generate_final_answer(state, llm))
    
    # Define edges
    workflow.add_edge("check_completeness", "retrieve_additional_info")
    workflow.add_edge("retrieve_additional_info", "generate_final_answer")
    workflow.add_edge("generate_final_answer", END)
    
    # Set entry point
    workflow.set_entry_point("check_completeness")
    
    return workflow

def solve_query_with_workflow(query: str, initial_response: str, groq_llm, vector_store_index: VectorStoreIndex) -> str:
    # Pass groq_llm as a keyword argument
    llm = GroqLLMWrapper(groq_llm=groq_llm)  
    retriever_wrapper = LlamaIndexVectorStoreRetriever(vector_store_index=vector_store_index, similarity_top_k=5)
    workflow = create_workflow(llm, retriever_wrapper)
    
    # Create the runnable graph
    app = workflow.compile()
    
    # Prepare the initial state
    initial_state = WorkflowState(
        query=query,
        initial_response=initial_response
    )
    
    # Run the workflow
    final_state = app.invoke(initial_state)
    
    return final_state.final_answer"""


In [19]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [20]:
from llama_index.vector_stores.chroma import ChromaVectorStore


In [20]:
# create client and a new collection
chroma_client = chromadb.EphemeralClient()
chroma_collection = chroma_client.create_collection("quickstart")


In [24]:
# set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [25]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)


In [26]:
# load from disk
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [31]:
"""def index_setup(nodes, embed_model, db_path="./chroma_db"):
    # Create or connect to the persistent ChromaDB
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")

    # Create ChromaVectorStore
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
    
    # Create storage context and index
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex(
        nodes, storage_context=storage_context, embed_model=embed_model
    )

    return index"""

In [ ]:
"""def load_index(embed_model, db_path="./chroma_db"):
    # Load the index from persistent storage
    db = chromadb.PersistentClient(path=db_path)
    chroma_collection = db.get_or_create_collection("quickstart")
    vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

    # Create the index from the vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store=vector_store,
        embed_model=embed_model
    )

    return index"""

In [32]:
#nodes_index = index_setup(nodes,embed_model, db_path="./chroma_db_nodes")


In [ ]:
"""query_engine = nodes_index.as_query_engine(llm=llm
    ,similarity_top_k=3, verbose=True
)

question1 = "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child."

response = query_engine.query(question1)
print(response.source_nodes)"""

In [ ]:
# load from disk , nodes 
"""db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("quickstart")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)"""

In [21]:
from llama_index.core import Settings


In [29]:
recursive_query_engine = index.as_query_engine(llm=llm
    ,similarity_top_k=5, verbose=True
)

In [ ]:
query = "Why is sound wave called a longitudinal wave?"
response = recursive_query_engine.query(query)
print(response)

In [ ]:
print(documents[-2].text[::])

In [ ]:
print(documents[-2].text[::])

In [45]:
questions = {
1 : "What is sound and how is it produced?",

2: "Describe with the help of a diagram, how compressions and rarefactions are produced in air near a source of sound.",

3: "Why is sound wave called a longitudinal wave?",

4: "Which characteristic of the sound helps you to identify your friend by his voice while sitting with others in a dark room?",

5: "Flash and thunder are produced simultaneously. But thunder is heard a few seconds after the flash is seen, why?",

6: "A person has a hearing range from 20 Hz to 20 kHz. What are the typical wavelengths of sound waves in air corresponding to these two frequencies? Take the speed of sound in air as 344 m s⁻¹.",

7: "Two children are at opposite ends of an aluminium rod. One strikes the end of the rod with a stone. Find the ratio of times taken by the sound wave in air and in aluminium to reach the second child.",

8: "The frequency of a source of sound is 100 Hz. How many times does it vibrate in a minute?",

9: "Does sound follow the same laws of reflection as light does? Explain.",

10: "When a sound is reflected from a distant object, an echo is produced. Let the distance between the reflecting surface and the source of sound production remains the same. Do you hear echo sound on a hotter day?",

11: "Give two practical applications of reflection of sound waves.",

12: "A stone is dropped from the top of a tower 500 m high into a pond of water at the base of the tower. When is the splash heard at the top? Given, g = 10 m s⁻² and speed of sound = 340 m s⁻¹.",

13: "A sound wave travels at a speed of 339 m s⁻¹. If its wavelength is 1.5 cm, what is the frequency of the wave? Will it be audible?",

14: "What is reverberation? How can it be reduced?",

15: "What is loudness of sound? What factors does it depend on?",

16: "How is ultrasound used for cleaning?",

17: "Explain how defects in a metal block can be detected using ultrasound."
}


In [38]:
answers = []

for value in questions.values(): 
    repo = recursive_query_engine.query(value)
    answers.append(repo)

In [ ]:
print(answers[6])

In [ ]:
questions.items()

In [51]:
ans = {}
for key, value in questions.items():
    ans[key] = recursive_query_engine.query(value)

In [ ]:
print(ans[14])